### Machine Learning 101: Intro to Neural Networks
#### Author: Alex - Super Make Something
#### Date: 2019-07-25
#### Supplemental YouTube video: Machine Learning 101: Intro to Neural Networks (Featured the NVIDIA Jetson Nano!)
#### Video URL: https://youtu.be/1BneqPdEhMM

<img src="img/autoencoderSchematic.png" alt="An autoencoder neural network" width="800">
<center> <i>An autoencoder neural network.</i> </center>

Machine learning is everywhere!  An artificial neural network (or "neural network" for short) is a type of machine learning algorithm that is particularly well suited for many applications because of its general-purpose structure.  But what exactly is a neural network and how can you set one up?  The code below implements sets up, trains, and tests an autoencoder neural network that compresses and decompresses images!

#### Import required libraries

This neural network is built using TensorFlow and Keras, two open-source machine learning libraries for Python.  Additionally, this code uses OpenCV2 (an image processing library) to load and save images, Numpy to manipulate image arrays, Matplotlib to plot images, Random to generate random numbers, and os to generate and create file paths and folders.

The cell below first imports all of the required libraries, and then prints out the version of the installed TensorFlow library to verify that everything installed correctly.  

Note: You can run individual Jupyter Notebook cells with SHIFT+ENTER.

In [1]:
#import required libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
import cv2 #OpenCV(cv2)
import os #other helper libraries
from os import path
import numpy as np
import matplotlib.pyplot as plt
import random

print(tf.__version__) #print tensorflow version

1.13.1


#### Define required supporting methods

The cell below defines the _loadAndResizeImages()_ method, which both loads the number of desired images defined by the _noImages_ argument, and then resizes it to a desired square size.  After this, it saves the image to the specified target directory.  A predefined random seed is used to "randomly" grab images from a folder in a repeatable way for testing/debugging purposes.

In [2]:
def loadAndResizeImages(noImages, desiredSize, targetDirectory, randomSeed): # Load and resize training images 
      
    if not os.path.exists(targetDirectory): #check for existence of target directory
            os.mkdir(targetDirectory) #create it if it doesn't exist
    
    filenames=os.listdir("images_raw") #Get filenames in "trainingImages" directory
    random.seed(randomSeed)
    for i in range(noImages):
        randIdx=random.randint(0,len(filenames))
        img=cv2.imread('images_raw/'+filenames[randIdx],1) #Load images in color
        #img=cv2.imread('images_raw/'+filenames[i],1) #Load images in color
        imDims=img.shape[:] #Image dimensions
        #print(img.shape)
        if imDims[0]<imDims[1]:
                imROI=img[0:imDims[0], 0:imDims[0],:]
        else:
                imROI=img[0:imDims[1], 0:imDims[1],:]
        #cv2.imshow("test", imROI) #Show extracted image
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        
        imResized=cv2.resize(imROI,(desiredSize,desiredSize)) #Resize image
        #cv2.imshow("test", imResized) #Show resized image
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        cv2.imwrite(targetDirectory+'/filename'+str(i)+'.jpg',imResized) #Save resized training images

The cell below performs prediction on the input data array, plots, and saves the results.

In [3]:
def plotResults(autoencoder, encoderNetwork,decoderNetwork,imageArray,desiredSize,targetDirectory):
    
    if not os.path.exists(targetDirectory): #check for existence of target directory
        os.mkdir(targetDirectory) #create it if it doesn't exist
    
    autoenc_imgs = autoencoder.predict(imageArray)
    encoded_imgs = encoderNetwork.predict(imageArray)
    decoded_imgs = decoderNetwork.predict(encoded_imgs)
    #decoded_imgs = autoencoder.predict(imageArray)

    n = 5  # how many images will be plotted
    totImgs=len(imageArray) # total images in array
    
    plt.figure(figsize=(n, n))
    
    for i in range(totImgs):
        
        imageDims=np.sqrt(encoded_imgs[i].size)

        if i<n:
            # display original
            ax = plt.subplot(3, n, i + 1)
            plt.imshow(imageArray[i].reshape(desiredSize, desiredSize))
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            
            # display encoded images
            ax = plt.subplot(3, n, i + 1 + n)
            plt.imshow(encoded_imgs[i].reshape(imageDims.astype(int), imageDims.astype(int)))
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
        
            # display autoencoder reconstruction
            ax = plt.subplot(3, n, i + 1 + 2*n)
            plt.imshow(decoded_imgs[i].reshape(desiredSize, desiredSize))
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
        
        #Reshape data "images"
        encodedImage=encoded_imgs[i].reshape(imageDims.astype(int), imageDims.astype(int));
        reconstructedImage=decoded_imgs[i].reshape(desiredSize, desiredSize);
        
        #Save data images
        plt.imsave(targetDirectory+'/encodedImage'+str(i)+'.png',encodedImage);
        plt.imsave(targetDirectory+'/reconstructedImage'+str(i)+'.png',reconstructedImage);
        
    plt.show()

With all of the function declarations complete, it's time to move onto the main program!  The below cell specifies the number of desired training and testing samples, as well as the desired input image size, and the number of iterations (or epochs) used to train the autoencoder neural network.

In [4]:
    noImagesTraining=2000 #Desired number of images to train on
    noImagesTesting=100 #Desired number of images to test on
    desiredSize=64 #Desired image size in pixels (square image)
    noEpochs=20000 

Next, the code loads and resizes the training and testing images, and saves them to their respective folders. It also initializes the array of training and testing images, extracts one color plane, and normalizes each pixel value to fall between 0-1, a typical range of values for the output of neural network nodes.

In [ ]:
    #Generate training data array
    targetDirectory="trainingImages"
    loadAndResizeImages(noImagesTraining, desiredSize,targetDirectory,0) # Load and resize training images 
    filenames=os.listdir(targetDirectory) #Get filenames in "trainingImages" directory
    
    for i in range(len(filenames)):
        img=cv2.imread(targetDirectory+'/'+filenames[i],1)
        
        try: trainingImages #check if first iteration (i.e. does trainingImages array exist)
        except NameError: trainingImages=None
        if trainingImages is None:
            trainingImages=np.empty([len(filenames),img.shape[0],img.shape[1]])# create trainingImages array and populate with first image
        
        trainingImages[i,:,:]=img[:,:,0]
    
    trainingImages=trainingImages/255 #Normalize values in trainingImages
    #print(trainingImages)
    
    #Generate testing data array
    targetDirectory="testingImages"
    loadAndResizeImages(noImagesTesting, desiredSize,targetDirectory,3) # Load and resize training images 
    filenames=os.listdir(targetDirectory) #Get filenames in "trainingImages" directory
    
    for i in range(len(filenames)):
        img=cv2.imread(targetDirectory+'/'+filenames[i],1)
        
        try: testingImages #check if first iteration (i.e. does trainingImages array exist)
        except NameError: testingImages=None
        if testingImages is None:
            testingImages=np.empty([len(filenames),img.shape[0],img.shape[1]])# create trainingImages array and populate with first image
        
        testingImages[i,:,:]=img[:,:,0]
    
    testingImages=testingImages/255 #Normalize values in testingImages
    #print(testingImages)

The training and testing image arrays are then reshaped so that each input image is stored in one row of a two-dimensional array.

In [ ]:
    trainingImages=trainingImages.reshape((len(trainingImages), np.prod(trainingImages.shape[1:])))
    testingImages=testingImages.reshape((len(testingImages), np.prod(testingImages.shape[1:])))
    
    trainingImages=trainingImages[0:noImagesTraining,:]
    testingImages=testingImages[0:noImagesTesting,:]
    
    #print(trainingImages)
    #print(testingImages)

Time to set up the neural network!  The code below declares an autoencoder neural network that has two hidden encode layers and two hidden decode layers, all of which use the "rectified linear units" (ReLU) activation function.  Additionally, the code also "splits the network in half," and declares separate encoding and decoding networks using the same structure.  These last two networks will later be used to encode and decode images, allowing us to compress and decompress new images.

In [ ]:
    # Network and training parameters
    #print(desiredSize)
    encoding_dim=(desiredSize**2)/8
    #print(encoding_dim)
    encoding_dim2=encoding_dim/2
    #print(encoding_dim2)
    
    #Construct full Autoencoder network
    input_img = Input(shape=(desiredSize*desiredSize,))
    encoded = Dense(encoding_dim, activation='relu')(input_img) #layer 1 (encoding)
    encoded2 = Dense(encoding_dim2, activation='relu')(encoded) #layer 2 (encoding)
    decoded1 = Dense(encoding_dim, activation='relu')(encoded2) #layer 3 (decoding)
    decoded = Dense(desiredSize*desiredSize, activation='sigmoid')(decoded1) #layer 4 (decoding) output

    autoencoder = Model(input_img, decoded) #Declare model
    autoencoder.summary()
    
    #Construct encoder network
    encoderNetwork = Model(autoencoder.input,autoencoder.layers[-3].output)
    encoderNetwork.summary()
    
    #Construct decoder network
    decoder_input = Input(shape=(encoding_dim2,))
    decoder_1 = autoencoder.layers[-2](decoder_input)
    decoder_2 = autoencoder.layers[-1](decoder_1)
    decoderNetwork = Model([decoder_input], decoder_2)
    decoderNetwork.summary()
    
    #autoencoder.compile(optimizer='adadelta', loss='mean_squared_error')
    autoencoder.compile(optimizer='adadelta', loss='logcosh')

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 4096)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               2097664   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               131584    
_________________________________________________________________
dense_3 (Dense)              (None, 4096)              2101248   
Total params: 4,461,824
Trainable params: 4,461,824
Non-trainable params: 0
_________________________________________________________________
____________________________________________________________

The cell below starts the neural network training process, and saves the resulting weights of both the overall auto-encoder network, and training/testing networks so that they can be reloaded later or on a different machine.

In [ ]:
    #Train the network
    trainingHistory=autoencoder.fit(trainingImages, trainingImages,
                epochs=noEpochs,
                batch_size=100,
                shuffle=True,
                verbose=1,
                validation_data=(testingImages, testingImages))
    
    #Save loss and validation loss for later plotting
    loss = trainingHistory.history['loss'] 
    val_loss = trainingHistory.history['val_loss']

    #Plot loss and validation loss after training
    plt.plot(loss)
    plt.plot(val_loss)
    plt.legend(['loss', 'val_loss'])
    plt.show()
    
    print("Saving Loss and Validation Loss...")
    np.save('loss',loss)
    np.save('val_loss',val_loss)        
    print("Saving Models...")
    autoencoder.save('autoencoder_large.h5')
    encoderNetwork.save('encoderNetwork_large.h5')
    decoderNetwork.save('decoderNetwork_large.h5')
    print("Saving Weights...")
    autoencoder.save_weights('weights_autoencoder_large.h5')
    encoderNetwork.save_weights('weights_encoderNetwork_large.h5')
    decoderNetwork.save_weights('weights_decoderNetwork_large.h5')
    print("Done!")
    
    print("Deleting models for testing purposes...")
    del autoencoder
    del encoderNetwork
    del decoderNetwork
    print("Done!")

Train on 2000 samples, validate on 100 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/20000
2000/2000 [==============================] - 2s 1ms/sample - loss: 0.0436 - val_loss: 0.0430
Epoch 2/20000
2000/2000 [==============================] - 2s 873us/sample - loss: 0.0435 - val_loss: 0.0430
Epoch 3/20000
2000/2000 [==============================] - 2s 910us/sample - loss: 0.0434 - val_loss: 0.0429
Epoch 4/20000
2000/2000 [==============================] - 2s 952us/sample - loss: 0.0434 - val_loss: 0.0429
Epoch 5/20000
2000/2000 [==============================] - 2s 1ms/sample - loss: 0.0433 - val_loss: 0.0429
Epoch 6/20000
2000/2000 [==============================] - 2s 918us/sample - loss: 0.0433 - val_loss: 0.0428
Epoch 7/20000
2000/2000 [==============================] - 2s 1ms/sample - loss: 0.0433 - val_loss: 0.0428
Epoch 8/20000
2000/2000 [==============================] - 2s 849us/sample - loss: 0.0433 - val_loss: 0.0428
Epoch 9/20000
2000/2000 [==============

Epoch 73/20000
2000/2000 [==============================] - 2s 765us/sample - loss: 0.0378 - val_loss: 0.0378
Epoch 74/20000
2000/2000 [==============================] - 2s 769us/sample - loss: 0.0378 - val_loss: 0.0378
Epoch 75/20000
2000/2000 [==============================] - 2s 764us/sample - loss: 0.0377 - val_loss: 0.0377
Epoch 76/20000
2000/2000 [==============================] - 2s 765us/sample - loss: 0.0376 - val_loss: 0.0377
Epoch 77/20000
2000/2000 [==============================] - 2s 768us/sample - loss: 0.0376 - val_loss: 0.0376
Epoch 78/20000
2000/2000 [==============================] - 2s 763us/sample - loss: 0.0375 - val_loss: 0.0376
Epoch 79/20000
2000/2000 [==============================] - 2s 761us/sample - loss: 0.0375 - val_loss: 0.0375
Epoch 80/20000
2000/2000 [==============================] - 2s 774us/sample - loss: 0.0374 - val_loss: 0.0374
Epoch 81/20000
2000/2000 [==============================] - 2s 767us/sample - loss: 0.0374 - val_loss: 0.0374
Epoch 82/2

2000/2000 [==============================] - 2s 805us/sample - loss: 0.0312 - val_loss: 0.0318
Epoch 148/20000
2000/2000 [==============================] - 2s 798us/sample - loss: 0.0311 - val_loss: 0.0316
Epoch 149/20000
2000/2000 [==============================] - 2s 812us/sample - loss: 0.0310 - val_loss: 0.0316
Epoch 150/20000
2000/2000 [==============================] - 2s 795us/sample - loss: 0.0309 - val_loss: 0.0314
Epoch 151/20000
2000/2000 [==============================] - 2s 796us/sample - loss: 0.0307 - val_loss: 0.0313
Epoch 152/20000
2000/2000 [==============================] - 2s 796us/sample - loss: 0.0306 - val_loss: 0.0312
Epoch 153/20000
2000/2000 [==============================] - 2s 806us/sample - loss: 0.0305 - val_loss: 0.0311
Epoch 154/20000
2000/2000 [==============================] - 2s 809us/sample - loss: 0.0304 - val_loss: 0.0310
Epoch 155/20000
2000/2000 [==============================] - 2s 801us/sample - loss: 0.0303 - val_loss: 0.0309
Epoch 156/20000
2

Epoch 221/20000
2000/2000 [==============================] - 2s 803us/sample - loss: 0.0241 - val_loss: 0.0237
Epoch 222/20000
2000/2000 [==============================] - 2s 806us/sample - loss: 0.0246 - val_loss: 0.0237
Epoch 223/20000
2000/2000 [==============================] - 2s 804us/sample - loss: 0.0241 - val_loss: 0.0236
Epoch 224/20000
2000/2000 [==============================] - 2s 806us/sample - loss: 0.0242 - val_loss: 0.0252
Epoch 225/20000
2000/2000 [==============================] - 2s 810us/sample - loss: 0.0244 - val_loss: 0.0263
Epoch 226/20000
2000/2000 [==============================] - 2s 797us/sample - loss: 0.0237 - val_loss: 0.0238
Epoch 227/20000
2000/2000 [==============================] - 2s 814us/sample - loss: 0.0242 - val_loss: 0.0245
Epoch 228/20000
2000/2000 [==============================] - 2s 807us/sample - loss: 0.0239 - val_loss: 0.0232
Epoch 229/20000
2000/2000 [==============================] - 2s 817us/sample - loss: 0.0238 - val_loss: 0.0241
E

Epoch 295/20000
2000/2000 [==============================] - 2s 809us/sample - loss: 0.0209 - val_loss: 0.0211
Epoch 296/20000
2000/2000 [==============================] - 2s 803us/sample - loss: 0.0209 - val_loss: 0.0204
Epoch 297/20000
2000/2000 [==============================] - 2s 803us/sample - loss: 0.0211 - val_loss: 0.0216
Epoch 298/20000
2000/2000 [==============================] - 2s 796us/sample - loss: 0.0202 - val_loss: 0.0203
Epoch 299/20000
2000/2000 [==============================] - 2s 805us/sample - loss: 0.0207 - val_loss: 0.0212
Epoch 300/20000
2000/2000 [==============================] - 2s 810us/sample - loss: 0.0210 - val_loss: 0.0201
Epoch 301/20000
2000/2000 [==============================] - 2s 871us/sample - loss: 0.0205 - val_loss: 0.0201
Epoch 302/20000
2000/2000 [==============================] - 2s 863us/sample - loss: 0.0206 - val_loss: 0.0211
Epoch 303/20000
2000/2000 [==============================] - 2s 907us/sample - loss: 0.0204 - val_loss: 0.0201
E

...

2000/2000 [==============================] - 2s 841us/sample - loss: 0.0089 - val_loss: 0.0106
Epoch 5201/20000
2000/2000 [==============================] - 2s 806us/sample - loss: 0.0089 - val_loss: 0.0106
Epoch 5202/20000
2000/2000 [==============================] - 2s 804us/sample - loss: 0.0089 - val_loss: 0.0106
Epoch 5203/20000
2000/2000 [==============================] - 2s 809us/sample - loss: 0.0089 - val_loss: 0.0106
Epoch 5204/20000
2000/2000 [==============================] - 2s 810us/sample - loss: 0.0089 - val_loss: 0.0106
Epoch 5205/20000
2000/2000 [==============================] - 2s 813us/sample - loss: 0.0089 - val_loss: 0.0107
Epoch 5206/20000
2000/2000 [==============================] - 2s 821us/sample - loss: 0.0090 - val_loss: 0.0107
Epoch 5207/20000
2000/2000 [==============================] - 2s 814us/sample - loss: 0.0089 - val_loss: 0.0106
Epoch 5208/20000
2000/2000 [==============================] - 2s 810us/sample - loss: 0.0089 - val_loss: 0.0106
Epoch 520

2000/2000 [==============================] - 2s 805us/sample - loss: 0.0089 - val_loss: 0.0105
Epoch 5347/20000
2000/2000 [==============================] - 2s 819us/sample - loss: 0.0088 - val_loss: 0.0105
Epoch 5348/20000
2000/2000 [==============================] - 2s 809us/sample - loss: 0.0088 - val_loss: 0.0105
Epoch 5349/20000
2000/2000 [==============================] - 2s 822us/sample - loss: 0.0088 - val_loss: 0.0105
Epoch 5350/20000
2000/2000 [==============================] - 2s 816us/sample - loss: 0.0088 - val_loss: 0.0105
Epoch 5351/20000
2000/2000 [==============================] - 2s 806us/sample - loss: 0.0088 - val_loss: 0.0105
Epoch 5352/20000
2000/2000 [==============================] - 2s 812us/sample - loss: 0.0088 - val_loss: 0.0105
Epoch 5353/20000
2000/2000 [==============================] - 2s 803us/sample - loss: 0.0088 - val_loss: 0.0105
Epoch 5354/20000
2000/2000 [==============================] - 2s 805us/sample - loss: 0.0088 - val_loss: 0.0106
Epoch 535

2000/2000 [==============================] - 2s 777us/sample - loss: 0.0087 - val_loss: 0.0105
Epoch 5493/20000
2000/2000 [==============================] - 2s 777us/sample - loss: 0.0087 - val_loss: 0.0105
Epoch 5494/20000
2000/2000 [==============================] - 2s 779us/sample - loss: 0.0087 - val_loss: 0.0104
Epoch 5495/20000
2000/2000 [==============================] - 2s 777us/sample - loss: 0.0087 - val_loss: 0.0105
Epoch 5496/20000
2000/2000 [==============================] - 2s 777us/sample - loss: 0.0087 - val_loss: 0.0106
Epoch 5497/20000
2000/2000 [==============================] - 2s 778us/sample - loss: 0.0088 - val_loss: 0.0105
Epoch 5498/20000
2000/2000 [==============================] - 2s 772us/sample - loss: 0.0088 - val_loss: 0.0105
Epoch 5499/20000
2000/2000 [==============================] - 2s 768us/sample - loss: 0.0088 - val_loss: 0.0105
Epoch 5500/20000
2000/2000 [==============================] - 2s 787us/sample - loss: 0.0087 - val_loss: 0.0105
Epoch 550

2000/2000 [==============================] - 2s 777us/sample - loss: 0.0087 - val_loss: 0.0105
Epoch 5639/20000
2000/2000 [==============================] - 2s 774us/sample - loss: 0.0087 - val_loss: 0.0104
Epoch 5640/20000
2000/2000 [==============================] - 2s 785us/sample - loss: 0.0086 - val_loss: 0.0104
Epoch 5641/20000
2000/2000 [==============================] - 2s 785us/sample - loss: 0.0087 - val_loss: 0.0105
Epoch 5642/20000
2000/2000 [==============================] - 2s 778us/sample - loss: 0.0087 - val_loss: 0.0105
Epoch 5643/20000
2000/2000 [==============================] - 2s 770us/sample - loss: 0.0087 - val_loss: 0.0104
Epoch 5644/20000
2000/2000 [==============================] - 2s 791us/sample - loss: 0.0087 - val_loss: 0.0104
Epoch 5645/20000
2000/2000 [==============================] - 2s 769us/sample - loss: 0.0087 - val_loss: 0.0104
Epoch 5646/20000
2000/2000 [==============================] - 2s 777us/sample - loss: 0.0087 - val_loss: 0.0104
Epoch 564

2000/2000 [==============================] - 2s 777us/sample - loss: 0.0086 - val_loss: 0.0104
Epoch 5785/20000
2000/2000 [==============================] - 2s 782us/sample - loss: 0.0086 - val_loss: 0.0104
Epoch 5786/20000
2000/2000 [==============================] - 2s 775us/sample - loss: 0.0086 - val_loss: 0.0104
Epoch 5787/20000
2000/2000 [==============================] - 2s 790us/sample - loss: 0.0086 - val_loss: 0.0104
Epoch 5788/20000
2000/2000 [==============================] - 2s 782us/sample - loss: 0.0086 - val_loss: 0.0104
Epoch 5789/20000
2000/2000 [==============================] - 2s 779us/sample - loss: 0.0086 - val_loss: 0.0104
Epoch 5790/20000
2000/2000 [==============================] - 2s 782us/sample - loss: 0.0086 - val_loss: 0.0105
Epoch 5791/20000
2000/2000 [==============================] - 2s 789us/sample - loss: 0.0086 - val_loss: 0.0104
Epoch 5792/20000
2000/2000 [==============================] - 2s 783us/sample - loss: 0.0086 - val_loss: 0.0104
Epoch 579

#### Encoding and decoding images with the trained network
Time to encode and decode some images with the trained networks!  The block of code below first reloads the trained autoencoder network and then extracts and compiles the encoder and decoder networks.

In [ ]:
    #Reload weights for additional testing/classification
    print("Reloading models for testing purposes...")
    autoencoder_reloaded=load_model('autoencoder_large.h5')
    encoderNetwork_reloaded=load_model('encoderNetwork_large.h5')
    decoderNetwork_reloaded=load_model('decoderNetwork_large.h5')
    print("Done!")

The below block of code inputs the array of training images and performs encoding/decoding with the trained network.  Each column corresponds to one image.  The first row of the plot is the original input image, the second row is the encoded image representation by the encoder network, and the third row is the recreated image from the encoded representation using the decoder network.  This code also saves the encoded and reconstructed output as separate images for later visualization outside of the Jupyter Notebook.

In [ ]:
    #Run autoencoder on training data
    plotResults(autoencoder_reloaded,encoderNetwork_reloaded,decoderNetwork_reloaded,trainingImages[0:noImagesTraining,:],desiredSize,'processedTrainingImages')

The next block of code does the same thing, but with a set of test images.

In [ ]:
#Run autoencoder on testing data
plotResults(autoencoder_reloaded,encoderNetwork_reloaded,decoderNetwork_reloaded,testingImages[0:noImagesTesting,:],desiredSize,'processedTestingImages')